In [20]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import librosa
import numpy as np

Extractor (Input Validation, Chunking, Mapping From file_csv and annotations_csv to chunks)

In [22]:
from acoustic_knowledge_discovery import extractors

In [23]:
extractor = extractors.Extractor(
    "/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/file.csv", #file metadata csv
    "/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/anno.csv", #annoations csv
    "/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/inputDtory",#input directory with a files subdirectory for audio files & a XC-templates subdirectory for XC templates
    chunk_size = 5) #chunk size in seconds
chunkDS = extractor.forward()


Casting the dataset: 100%|██████████| 6/6 [00:00<00:00, 881.03 examples/s]


In [24]:
for row in chunkDS.chunk_ds["train"]:
    print(row)
chunkDS.chunk_ds

{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'chunk_start': 0, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_0', 'Annotation': ['owl'], 'Confidence': [0.8999999761581421]}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'chunk_start': 5, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_5', 'Annotation': ['owl'], 'Confidence': [0.8999999761581421]}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'chunk_start': 10, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_10', 'Annotation': [], 'Confidence': []}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 's

DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence'],
        num_rows: 6
    })
})

Chunk DS Augmentations

In [25]:
def add_raw_audio_batched(batch):
    raws = []
    for fname, start in zip(batch["file_name"], batch["chunk_start"]):
        y, _ = librosa.load(
            extractor.base_dir / "files" / fname,
            offset=float(start),
            duration=float(extractor.chunk_size),
            sr=None
        )
        raws.append(y.astype(np.float32))
    # return list-of-arrays (Arrow will store as list<primitive>)
    return {"raw_audio": raws}


In [ ]:
# Add raw audio column in dataset
chunkDS.chunk_ds = chunkDS.chunk_ds.map(add_raw_audio_batched, 
                                        batched=True, 
                                        num_proc=4)

Map (num_proc=4): 100%|██████████| 6/6 [00:00<00:00, 18.58 examples/s]


In [27]:
chunkDS.chunk_ds #make sure that new column raw_audio is added

DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'raw_audio'],
        num_rows: 6
    })
})

File-level Operations

In [28]:
from acoustic_knowledge_discovery.features.feature_preprocessing_pipeline import FeatureSequential
from acoustic_knowledge_discovery import features

THRESHOLD = 0.6

pipeline = FeatureSequential(
    features.TemplateMatching(
        extractor.base_dir / "files",
        extractor.base_dir / "XC-templates",
        THRESHOLD,
        chunk_size=extractor.chunk_size
    ),
    # features.anotherFunctionThatInheretsFromFeaturePreprocessor
)

chunkDS = pipeline.forward(chunkDS)  

1 total templates
Filtered down to 1 unique species templates
  XC758442 - Common Blackbird - Turdus merula: XC758442 - Common Blackbird - Turdus merula.wav


Map: 100%|██████████| 6/6 [00:00<00:00, 370.69 examples/s]

Progress: 1/1 templates completed (100.0%)
Total matches found above the threshold of 0.6: 1


In [29]:
#verify properly loaded after feature extraction
for row in chunkDS.chunk_ds["train"]:
    print(row["Annotation"], row["Confidence"])

['owl'] [0.8999999761581421]
['owl'] [0.8999999761581421]
[] []
[] []
['XC758442 - Common Blackbird - Turdus merula'] [0.6061245203018188]
['bat', 'XC758442 - Common Blackbird - Turdus merula'] [0.800000011920929, 0.6061245203018188]


Chunk-level Operations

In [35]:
#Insert EGCI
egci = features.EGCI()
chunkDS= egci(chunkDS)


EGCI __init__
EGCI __call__


Map (num_proc=4): 100%|██████████| 6/6 [00:08<00:00,  1.43s/ examples]


In [ ]:
#verify EGCI column properly added
for row in chunkDS.chunk_ds["train"]:
    print(row["EGCI"])
chunkDS.chunk_ds

[0.6825488342933071]
[0.6520874520975383]
[0.6374639176949186]
[0.5778173748528687]
[0.7267189092284931]
[0.8713889758341212]


DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'raw_audio', 'EGCI'],
        num_rows: 6
    })
})

Canonical Graph Representation